In [8]:
%config InteractiveShell.ast_node_interactivity = 'all' # 'all', 'last', 'last_expr' or 'none', 'last_expr_or_assign'
# %config InlineBackend.figure_format = 'svg'


In [2]:
import numpy as np
import gym
from envs.cleanup import CleanupEnv
from envs.cleanup import CleanupAgent

# self, agent_id, start_pos, start_orientation, grid, row_size, col_size

env = 'cleanup-v0'
try:
    gym.register(env, entry_point=CleanupEnv)
except:
    raise Exception('environment registered')
env = gym.make(env)



In [2]:
from envs.harvest import HarvestEnv
from envs.cleanup import CleanupEnv
from algo.ppo import PPO
from algo.model import Policy
from algo.storage import RolloutStorage
# from config import get_config
from envs.vec_env import SubprocVecEnv, DummyVecEnv
from algo.algo_utils import update_linear_schedule

In [3]:
def make_parallel_env(args):
    def get_env_fn(rank):
        def init_env():
            if args['env_name'] == "cleanup":
                env = CleanupEnv(num_agents=args['num_agents'])
            elif args['env_name'] == "harvest":
                env = HarvestEnv(num_agents=args['num_agents'])
            else:
                print("Can not support the " + args['env_name'] + "environment." )
                raise NotImplementedError
            env.seed(args['seed'] + rank * 1000)
            return env
        return init_env
    if args['n_rollout_threads'] == 1:
        return DummyVecEnv([get_env_fn(0)])
    else:
        return SubprocVecEnv([get_env_fn(i) for i in range(args['n_rollout_threads'])])

In [4]:
args = {'env_name':'cleanup', 'seed':2, 'n_rollout_threads':4, 'num_agents': 2}
envs = make_parallel_env(args)

In [5]:
obs = envs.reset()
obs.shape


(4, 2, 15, 15, 3)

In [7]:
import numpy as np
a = np.arange(960).reshape(10,2,3,4,4)
c = np.concatenate([a[:,i,:,:,:] for i in range(2)], axis=1)
c.shape


(10, 6, 4, 4)

In [12]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
writer = SummaryWriter()
for i in range(10):
    x = np.arange(4)
    np.random.shuffle(x)
    writer.add_histogram('distribution centers', x , i)
# writer.close()

In [13]:
! tensorboard --logdir=/home/liyunfei/lsy/ssd/runs --port=9999

TensorBoard 1.15.0 at http://h1:9999/ (Press CTRL+C to quit)
^C


In [ ]:
env = CleanupEnv(num_agents=4)